# **Part 1: Text Preprocessing**



**Data-** given in the below code cell

**1.1: Preprocessing From Scratch**

**Goal:** Write a function clean_text_scratch(text) that performs the following without using NLTK or Spacy:

1. Lowercasing: Convert text to lowercase.

2. Punctuation Removal: Use Python's re (regex) library or string methods to remove special characters (!, ., ,, :, ;, ..., ').

3. Tokenization: Split the string into a list of words based on whitespace.

4. Stopword Removal: Filter out words found in this list: ['the', 'is', 'in', 'to', 'of', 'and', 'a', 'it', 'was', 'but', 'or'].

5. Simple Stemming: Create a helper function that removes suffixes 'ing', 'ly', 'ed', and 's' from the end of words.


Note: This is a "Naive" stemmer. It will break words like "sing" -> "s". This illustrates why we need libraries!

**Task:** Run this function on the first sentence of the corpus and print the result.

In [ ]:
corpus = [
    "Artificial Intelligence is transforming the world; however, ethical concerns remain!",
    "The pizza was absolutely delicious, but the service was terrible ... I won't go back.",
    "The quick brown fox jumps over the lazy dog.",
    "To be, or not to be, that is the question: Whether 'tis nobler in the mind.",
    "Data science involves statistics, linear algebra, and machine learning.",
    "I love machine learning, but I hate the math behind it."
]

In [ ]:
import re
#write rest of the code here
stopwords_basic = {'the', 'is', 'in', 'to', 'of', 'and', 'a', 'it', 'was', 'but', 'or'}
def simple_lemmatizer(word):
    if word.endswith("ing"):
        return word[:-3]
    if word.endswith("ed"):
        return word[:-2]
    if word.endswith("s") and len(word) > 3:
        return word[:-1]
    return word
def clean_text_scratch(text):
  text = text.lower()
  text = re.sub(r'[^\w\s]', '', text)
  tokens = text.split()
  tokens = [w for w in tokens if w not in stopwords_basic]
  tokens = [simple_lemmatizer(w) for w in tokens]
  return tokens
print(clean_text_scratch(corpus[0]))

**1.2: Preprocessing Using Tools**

**Goal:** Use the nltk library to perform the same cleaning on the entire corpus.

**Steps:**

1. Use nltk.tokenize.word_tokenize.
2. Use nltk.corpus.stopwords.
3. Use nltk.stem.WordNetLemmatizer

to convert words to their root (e.g., "jumps" $\to$ "jump", "transforming" $\to$ "transform").


**Task:** Print the cleaned, lemmatized tokens for the second sentence (The pizza review).

In [ ]:
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

#write rest of the code here
stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()
def clean_text_nltk(text):
    text = text.lower()
    text = text.translate(str.maketrans("", "", string.punctuation))
    tokens = word_tokenize(text)
    tokens = [w for w in tokens if w not in stop_words]
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    return tokens
print(clean_text_nltk(corpus[1]))

# **Part 2: Text Representation**

**2.1: Bag of Words (BoW)**

**Logic:**

**Build Vocabulary:** Create a list of all unique words in the entire corpus (after cleaning). Sort them alphabetically.

**Vectorize:** Write a function that takes a sentence and returns a list of numbers. Each number represents the count of a vocabulary word in that sentence.

**Task:** Print the unique Vocabulary list. Then, print the BoW vector for: "The quick brown fox jumps over the lazy dog."

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# NLTK downloads (run once)
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# rest of the code here
def bow_scratch(corpus):
  tokenized = [clean_text_nltk(text) for text in corpus]
  vocab = sorted(list(set([word for text in tokenized for word in text])))
  bow_matrix = []
  for text in tokenized:
    bow_vector = [text.count(word) for word in vocab]
    bow_matrix.append(bow_vector)
  return vocab, bow_matrix
vocab, bow_matrix = bow_scratch(corpus)
print("Vocabulary:", vocab)
print("BoW vector for \"The quick brown fox jumps over the lazy dog.\":")
print(bow_matrix[2])

**2.2: BoW Using Tools**

**Task:** Use sklearn.feature_extraction.text.CountVectorizer.

**Steps:**

1. Instantiate the vectorizer.

2. fit_transform the raw corpus.

3. Convert the result to an array (.toarray()) and print it.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
#rest of the code here
vectorizer = CountVectorizer()
bow_matrix = vectorizer.fit_transform(corpus)
bow_array = bow_matrix.toarray()
print("Vocabulary:")
print(vectorizer.get_feature_names_out())
print("BoW Matrix:")
print(bow_array)

**2.3: TF-IDF From Scratch (The Math)**

**Goal:** Manually calculate the score for the word "machine" in the last sentence:

"I love machine learning, but I hate the math behind it."

**Formula:**

*TF (Term Frequency):* $\frac{\text{Count of 'machine' in sentence}}{\text{Total words in sentence}}$

*IDF (Inverse Document Frequency):* $\log(\frac{\text{Total number of documents}}{\text{Number of documents containing 'machine'}})$ (Use math.log).

**Result:** TF * IDF.

**Task:** Print your manual calculation result.

In [ ]:
# write code here
import math
target_word = "machine"
sentence = corpus[5]
tokens = sentence.lower().split()
total_words = len(tokens)
word_count = sum(1 for w in tokens if w.strip(",.!?;-") == target_word)
TF = word_count / total_words
N = len(corpus)
df = 0
for doc in corpus:
    doc_tokens = [w.strip(",.!?;-").lower() for w in doc.split()]
    if target_word in doc_tokens:
        df += 1
IDF = math.log(N / df)
TF_IDF = TF * IDF
print("TF:", TF)
print("IDF:", IDF)
print("TF-IDF:", TF_IDF)

**2.4: TF-IDF Using Tools**

**Task:** Use sklearn.feature_extraction.text.TfidfVectorizer.

**Steps:** Fit it on the corpus and print the vector for the first sentence.

**Observation:** Compare the score of unique words (like "Intelligence") vs common words (like "is"). Which is higher?

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# rest of the code here
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(corpus)
tfidf_array = tfidf_matrix.toarray()
print("Vocabulary:")
print(tfidf.get_feature_names_out())
print("TF-IDF vector for corpus[0]:")
print(tfidf_array[0])

# **Part 3- Word Embeddings**

**3.1: Word2Vec Using Tools**

**Task:** Train a model using gensim.models.Word2Vec.

**Steps:**

1. Pass your cleaned tokenized corpus (from Part 1.2) to Word2Vec.

2. Set min_count=1 (since our corpus is small, we want to keep all words).

3. Set vector_size=10 (small vector size for easy viewing).

**Experiment:** Print the vector for the word "learning".

In [ ]:
!pip install gensim
from gensim.models import Word2Vec
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

#rest of the code here
cleaned_corpus = [clean_text_nltk(text) for text in corpus]
model = Word2Vec(sentences=cleaned_corpus, min_count=1, vector_size=10)
print("Vector for 'learning':")
print(model.wv["learning"])

**3.3: Pre-trained GloVe (Understanding Global Context)**

**Task:** Use gensim.downloader to load 'glove-wiki-gigaword-50'

**Analogy Task:** Compute the famous analogy:$\text{King} - \text{Man} + \text{Woman} = ?$

Use model.most_similar(positive=['woman', 'king'], negative=['man']).

**Question:** Does the model correctly guess "Queen"?

In [ ]:
import gensim.downloader as api

# Load pre-trained GloVe model
glove_model = api.load('glove-wiki-gigaword-50')

#rest of the code here
result = glove_model.most_similar(positive=['woman', 'king'], negative=['man'])
print("Analogy result:")
print(result[0][0])
#Yes, the model correctly guesses "queen".

# **Part 5- Sentiment Analysis (The Application)**

**Concept:** Sentiment Analysis determines whether a piece of text is Positive, Negative, or Neutral. We will use VADER (Valence Aware Dictionary and sEntiment Reasoner) from NLTK. VADER is specifically designed for social media text; it understands that capital letters ("LOVE"), punctuation ("!!!"), and emojis change the sentiment intensity.

**Task:**

1. Initialize the SentimentIntensityAnalyzer.

2. Pass the Pizza Review (corpus[1]) into the analyzer.

3. Pass the Math Complaint (corpus[5]) into the analyzer.

**Analysis:** Look at the compound score for both.

**Compound Score Range:** -1 (Most Negative) to +1 (Most Positive).

Does the model correctly identify that "delicious" and "terrible" in the same sentence result in a mixed or neutral score?

In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
# Download VADER (run once)
nltk.download('vader_lexicon')
# rest of the code here
sia = SentimentIntensityAnalyzer()
pizza_review = corpus[1]
math_complaint = corpus[5]
score_pizza = sia.polarity_scores(pizza_review)
score_math = sia.polarity_scores(math_complaint)
print("Pizza Review Sentiment:", score_pizza)
print("Math Complaint Sentiment:", score_math)
print("\nObservation:")
print("Pizza compound score:", score_pizza['compound'])
print("Math compound score:", score_math['compound'])
#Although both sentences contain both positive and negative expressions, VADER assigns a more negative compound score because negative sentiment words such as “terrible”, “won’t go back”, and “hate” carry higher intensity. Therefore, the final sentiment is negative rather than perfectly neutral.